In [2]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import requests

In [3]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/train.json?TWGServiceId=kakao_arena&Expires=1625698782&Signature=VgPWMbIQ0F11kfIMmnTXHEErDCI%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
train=r.json()

train_df=pd.DataFrame(train)
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [4]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/genre_gn_all.json?TWGServiceId=kakao_arena&Expires=1625698829&Signature=rxsNMoZeURLESdUblZbMlnl6WVs%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
genre=r.json()

genre_ser=pd.Series(genre)
genre_ser

GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
           ...  
GN2900       뮤지컬
GN2901    세부장르전체
GN2902     국내뮤지컬
GN2903     국외뮤지컬
GN3000     크리스마스
Length: 254, dtype: object

In [5]:
url="https://arena.kakaocdn.net/kakao_arena/melon_autoplaylist/song_meta.json?TWGServiceId=kakao_arena&Expires=1625698850&Signature=y59jYMXifv4h0iwct4oi6puAcrE%3D&AllowedIp=1.246.85.102&download"
r=requests.get(url)
r.encoding="utf-8"
song=r.json()

song_df=pd.DataFrame(song)
song_df.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [6]:
train_songs=[]
for i in train:
    train_songs.append(i["songs"])
    
print(train_songs[:2],"\n")
print(len(train_songs))

[[525514, 129701, 383374, 562083, 297861, 139541, 351214, 650298, 531057, 205238, 706183, 127099, 660493, 461973, 121455, 72552, 223955, 324992, 50104], [432406, 675945, 497066, 120377, 389529, 244277, 461062, 696302, 442765, 532114, 586541, 33389, 244000, 692078, 37741, 645653, 571802, 200183, 61435, 204499, 41749, 129258, 413920, 117205, 6546, 152422, 602724, 425946, 173634, 631268, 409869, 97749, 395416, 103741, 181101, 472144, 414721, 75801, 315216, 192882, 383960, 548636]] 

115071


##### Word2Vec 
  : 단어마다 차례대로 인덱싱을 하여 벡터화하지 않고,
    유사한 단어들을 비슷한 방향과 힘의 벡터를 갖도록 단어를 벡터화 시켜주는 방법 중 하나
 
+  Word2Vec의 두가지 방법
 1. CBOW (Coutinuous Bag of Word)
     - 주변 단어를 통해 중심 단어를 예측하는 방식
 2. Skin-gram
     - 중심 단어를 통해 주변 단어를 예측하는 방식
 <img src="./image/cbow skip-gram.png" height="500px" width="500px" >
 + Word2Vec의 형태\
 model=word2vec.Word2Vec(sentences, workers, size, min_count, window, sample,sg,iter)
     1. **sentences** : 학습시킬 문장
     2. **workers** : 실행할 병렬 프로세스의 수
     3. **size** : 각 단어에 대한 임베딩 된 벡터차원 정의\
         -> size=2라면 문장의 벡터는[-0.1248574, 0.255778]와 같은 형태로 가지게 된다
     4. **min_count** : 단어에 대한 최소 빈도수 \
         -> min_count=5라면 빈도수 5이하는 무시
     5. **sample** : 빠른 학습을 위해 정답 단어 라벨에 대한 다운 샘플링 비율을 지정하는 것,\
           보통 0.001이 좋은 성능을 낸다고 한다
     6. **sg** : 1이면 skip-gram, 0이면 CBOW
     7. **iter** : epoch와 같은 뜻으로 학습 반복 횟수 지정
     8.  window : 중심 단어를 예측하기 위해 앞,뒤로 몇개의 단어를 포함할 것인지의 범위를 window 라고한다
+ 모델 교육을 3단계로 구분하는 이유¶
    - 명확성과 모니터링을 위해 교육을 세 단계로 구분하는 것이 좋습니다.
    1. Word2Vec()\
        : 이 첫 번째 단계에서는 모델의 파라미터를 하나씩 설정합니다.\
          매개 변수 문장을 제공하지 않기 때문에 모델을 초기화하지 않은 상태로 일부러 둡니다.
    2. build_vocab()\
        : 여기 문장의 시퀀스에서 모델 초기화된 어휘들을 만든다.
    3. train()\
        : 마지막으로 모델을 train합니다.

- Word2Vec 프로세스

<img src="./image/Word2Vec2.png" height="650px" width="650px" >


1.어휘 빌더\
    : 문장 형태의 원시 데이터를 가져와 고유 한 단어를 추출하여 시스템의 어휘를 구축한다\
2. 컨텍스트 빌더\
    : 단어를 벡터화한다\
        (어휘 빌더의 출력, 즉 색인과 개수를 입력으로 포함하는 어휘 객체)

In [7]:
from gensim.models import Word2Vec

# Word2Vec 훈련 - 대규모 곡 리스트에서 곡끼리의 연관성을 학습
item2vec=Word2Vec(vector_size=300,window=7,min_count=1,workers=10,sg=1)
# 곡 사전 빌더
songs=list(range(len(song)))
item2vec.build_vocab([songs])
# 학습 진행 
item2vec.train(corpus_iterable=train_songs,epochs=10,total_words=len(song),total_examples=1)

(52858710, 52858710)

In [27]:
# Word2Vec 모델 저장
item2vec.save("./item/item2vec.model2")

In [8]:
# Word2Vec 모델 로드
item2vec=Word2Vec.load("./item/item2vec.model2")
item2vec

In [9]:
# 유사한 곡 10개 추출
top10=item2vec.wv.most_similar(positive=[152422],topn=10)
top10

[(232606, 0.8481897115707397),
 (144663, 0.8442291021347046),
 (224921, 0.8393559455871582),
 (351342, 0.8349840641021729),
 (357367, 0.8343160152435303),
 (77951, 0.8335286378860474),
 (339118, 0.8333936929702759),
 (174749, 0.8332453966140747),
 (171798, 0.83250892162323),
 (118788, 0.8307193517684937)]

In [10]:
# 학습 데이터 생성????????????????
import random

songs_map={s["id"]:s for s in song}

train_set=[]

for i in train:
    songs=i["songs"]
    tags=i["tags"]
    gnrs=[]
    for j in songs:
        gnrs.extend(songs_map[j]["song_gn_dtl_gnr_basket"])
    gnrs=set(gnrs)
    
    if len(gnrs)<5:
        continue
        
    sample_size=len(songs)//7
    
    for j in range(sample_size):
        npl={}
        
        rs=random.sample(songs,7)
        
        npl["s1"]=rs[0]
        npl["s2"]=rs[1]
        npl["s3"]=rs[2]
        npl["s4"]=rs[3]
        npl["s5"]=rs[4]
        npl["s6"]=rs[5]
        npl["label"]=rs[6]
        
        rg=random.sample(gnrs,5)
        
        npl["g1"]=rg[0]
        npl["g2"]=rg[1]
        npl["g3"]=rg[2]
        npl["g4"]=rg[3]
        npl["g5"]=rg[4]
        
        if len(tags)>3:
            
            rt=random.sample(tags,3)
            
            npl["t1"]=rt[0]
            npl["t2"]=rt[1]
            npl["t3"]=rt[2]
            
        elif len(tags)==0:
            npl["t1"]=""
            npl["t2"]=""
            npl["t3"]=""
            
        elif len(tags)==1:
            npl["t1"]=tags[0]
            npl["t2"]=""
            npl["t3"]=""
            
        elif len(tags)==2:
            npl["t1"]=tags[0]
            npl["t2"]=tags[1]
            npl["t3"]=""
        
        train_set.append(npl)
train_df=pd.DataFrame(train_set)
train_df

,s1,s2,s3,s4,s5,s6,label,g1,g2,g3,g4,g5,t1,t2,t3
0,297861,562083,660493,205238,531057,223955,139541,GN1304,GN1302,GN1001,GN1003,GN1101,락,,
1,461973,351214,72552,660493,129701,205238,525514,GN1901,GN0904,GN1003,GN1301,GN0902,락,,
2,425946,395416,33389,602724,442765,675945,129258,GN0104,GN1502,GN0501,GN0103,GN0805,추억,회상,
3,692078,442765,461062,696302,244277,413920,409869,GN1803,GN0605,GN0901,GN1001,GN1608,추억,회상,
4,61435,41749,152422,586541,200183,461062,204499,GN0601,GN1706,GN2601,GN1001,GN2602,추억,회상,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681125,627286,313958,405465,567849,105350,3179,459334,GN0907,GN1108,GN1013,GN1304,GN1402,노래추천,팝송모음,팝송
681126,535146,407370,690767,700937,676563,507928,430481,GN0908,GN1002,GN0902,GN1004,GN1013,팝송,노래추천,팝송모음
681127,185139,314714,544251,381746,211723,567849,701978,GN1104,GN1202,GN1506,GN1101,GN1005,팝송,팝송모음,팝송추천
681128,374554,439642,535146,192295,117852,207558,695590,GN1303,GN1108,GN1201,GN1404,GN1202,팝송,팝송모음,노래추천


In [52]:
# 학습 데이터 저장??????????????????
train_df.to_csv("./keras_train.csv",index=False)

In [11]:
import KerasModel
import pandas as pd

# 학습 데이터 로드?????????????
train_df=pd.read_csv("./keras_train.csv")
train_df

,s1,s2,s3,s4,s5,s6,label,g1,g2,g3,g4,g5,t1,t2,t3
0,461973,205238,139541,383374,72552,50104,297861,GN1302,GN0901,GN1013,GN0902,GN1005,락,NaN,NaN
1,121455,650298,50104,461973,660493,525514,139541,GN1901,GN1301,GN0902,GN1304,GN1001,락,NaN,NaN
2,602724,6546,432406,97749,75801,497066,425946,GN0508,GN0502,GN0503,GN0105,GN0901,추억,회상,NaN
3,33389,120377,571802,173634,432406,425946,548636,GN0103,GN0105,GN0606,GN0101,GN1803,추억,회상,NaN
4,532114,461062,192882,414721,409869,645653,117205,GN1502,GN0103,GN1504,GN1708,GN1801,추억,회상,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681125,90675,18140,361959,255937,323543,676925,418210,GN1202,GN1307,GN1005,GN1013,GN1008,팝송모음,노래추천,팝송추천
681126,376372,283606,180979,32880,570768,117852,239015,GN1201,GN1503,GN1012,GN1407,GN2704,노래추천,팝송,팝송모음
681127,640239,621746,535751,185139,381746,324819,389791,GN1504,GN0901,GN1012,GN0908,GN1307,노래추천,팝송,팝송추천
681128,443713,73260,528972,600969,448331,18140,180979,GN1013,GN0907,GN1301,GN1012,GN1302,노래추천,팝송,팝송추천


In [12]:
train_df.isnull().sum()

s1            0
s2            0
s3            0
s4            0
s5            0
s6            0
label         0
g1            0
g2            0
g3            0
g4            0
g5            0
t1        98139
t2       243064
t3       365155
dtype: int64

In [13]:
train_df[["t1","t2","t3"]]=train_df[["t1","t2","t3"]].fillna("-")

songs_size=len(song)

gr_nq=list(genre)

t1_nq=train_df["t1"].unique()
t2_nq=train_df["t2"].unique()
t3_nq=train_df["t3"].unique()
tg_nq=np.concatenate((t1_nq,t2_nq,t3_nq))
tg_nq=np.unique(tg_nq)

# KerasModel 모델 훈련 - test data의 곡들과 유사한 곡들을 추천해 주기 위한 model????????
model=KerasModel.get_model(songs_size,gr_nq,tg_nq)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tags (InputLayer)               [(None, 3)]          0                                            
__________________________________________________________________________________________________
genrs (InputLayer)              [(None, 5)]          0                                            
__________________________________________________________________________________________________
songs (InputLayer)              [(None, 6)]          0                                            
__________________________________________________________________________________________________
string_lookup (StringLookup)    (None, 3)            0           tags[0][0]                       
______________________________________________________________________________________________

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

# 입력 데이터 변환 
sg_train=train_df[["s1","s2","s3","s4","s5","s6"]].to_numpy()
gr_train=train_df[["g1","g2","g3","g4","g5"]].to_numpy()
tg_train=train_df[["t1","t2","t3"]].to_numpy()

y_train=train_df[["label"]].to_numpy()

# 콜백함수 생성
mc=ModelCheckpoint("./model/search",moniter="val_loss",save_best_only=True,save_weights_only=True,mode="auto")

# 모델 학습
model.fit({"songs":sg_train,"tags":tg_train,"genrs":gr_train},
         y_train,
         batch_size=256,
         epochs=7,
         callbacks=[mc],
         validation_split=0.2)

Epoch 1/7
2129/2129 [==============================] - 2548s 1s/step - loss: 12.2007 - val_loss: 11.4270
Epoch 2/7
2129/2129 [==============================] - 2784s 1s/step - loss: 10.8441 - val_loss: 11.3899
Epoch 3/7
2129/2129 [==============================] - 2866s 1s/step - loss: 10.3423 - val_loss: 11.4007
Epoch 4/7
2129/2129 [==============================] - 2873s 1s/step - loss: 9.8835 - val_loss: 11.4558
Epoch 5/7
2129/2129 [==============================] - 2975s 1s/step - loss: 9.4224 - val_loss: 11.6377
Epoch 6/7
2129/2129 [==============================] - 2913s 1s/step - loss: 8.9792 - val_loss: 11.8788
Epoch 7/7
2129/2129 [==============================] - 2898s 1s/step - loss: 8.5832 - val_loss: 12.1807


In [15]:
# 모델 저장
model.save_weights("./model/searchover")

In [19]:
import random

# 모델 재평가????????????
model.load_weights("./model/search")

# 테스트 데이터 생성???????????????
songs_map={s["id"]:s for s in song}

for i in train[:10]:
    songs=i["songs"]
    tags=i["tags"]
    gnrs=[]
    for j in songs:
        gnrs.extend(songs_map[j]["song_gn_dtl_gnr_basket"])
    gnrs=set(gnrs)
    
    if len(gnrs)<5 or len(songs)<7:
        continue
        
    sg_test=random.sample(songs,6)
    
    gr_test=random.sample(gnrs,5)
    
    tg_test=["-","-","-"]
    if len(tags)>3:
        tg_test=random.sample(tags,3)
    else:
        for ind,t in enumerate(tags):
            tg_test[ind]=t
    tg_test=np.array(tg_test)
    
    # 모델 예측 
    psongs=model.predict({"songs":np.array([sg_test]),"tags":np.array([tg_test]),"genrs":np.array([gr_test])})
    print("psong : ",psongs[0][:5])
    # ???????????????
    psings=np.argsort(psongs[0])
    print("psings :",psings[:5])
    # ???????????????
    acc=[p for p in psings if p in songs]
    print("acc : ",acc[:5])
    print()
    

psong :  [-3.4443333 -3.4482734 -3.4491978 -3.448648  -3.4476728]
psings : [565427  64502 411639  39996  95525]
acc :  [525514, 351214, 50104, 383374, 562083]

psong :  [-6.934308  -6.9440136 -6.945582  -6.945429  -6.9440174]
psings : [ 64502 565427 228948 411639 629439]
acc :  [442765, 103741, 631268, 120377, 129258]

psong :  [-6.863863  -6.868536  -6.8698277 -6.870951  -6.869277 ]
psings : [ 64502 228948 411639 565427 629439]
acc :  [233195, 153029, 623704, 556426, 300104]

psong :  [-4.846544  -4.8482428 -4.8477383 -4.848384  -4.8482065]
psings : [ 55841 569864 647068 349955  87086]
acc :  [273672, 287984, 243754, 25155, 100335]

psong :  [-7.245043  -7.252428  -7.2533565 -7.2549257 -7.254265 ]
psings : [ 64502 565427 262405 228948  39996]
acc :  [503552, 487106, 146266, 359101, 324136]

psong :  [-8.6024    -8.6127205 -8.614422  -8.610804  -8.609705 ]
psings : [228948  95525 411639 313932 487972]
acc :  [307985, 231897, 263364, 643573, 559269]

psong :  [-7.6672354 -7.671628  -7.6

In [ ]:
# https://medium.com/@vishwasbhanawat/the-architecture-of-word2vec-78659ceb6638